In [2]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [5]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [6]:
type(train_dataset)

torchvision.datasets.mnist.MNIST

In [16]:
max(train_dataset.targets).item()

9

In [17]:
min(train_dataset.targets).item()

0

In [8]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [33]:
dik.values()

dict_values(['five', 'three', 'two', 'four', 'six', 'seven', 'eight', 'nine'])

In [231]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes, arch):
        super().__init__()
        self.device = device
        layerss = []
        sizes = []
        for l, v in arch:
            layerss.append(l)
            sizes.append(v)
        
        self.layers = nn.ModuleList()
        prev_size = input_size
        
        # Input layer
        size = sizes[0]
        self.layers.append(nn.Linear(in_features=prev_size, out_features=size).to(self.device))
        
        # Hidden layers
        prev_size = size
        for i in range(1, len(arch)):
            size = sizes[i]
            layer = layerss[i]
            if layer == 'linear':
                self.layers.append(nn.Linear(in_features=prev_size, out_features=size).to(self.device))
            elif layer == 'relu':
                self.layers.append(nn.ReLU().to(self.device))
            elif layer == 'sigmoid':
                self.layers.append(nn.Sigmoid().to(self.device))
            elif layer == 'batchnorm1d':
                self.layers.append(nn.BatchNorm1d(prev_size).to(self.device))
            elif layer == 'dropout':
                self.layers.append(nn.Dropout().to(self.device))
            elif layer == 'flatten':
                self.layers.append(nn.Flatten().to(self.device))
            elif layer == 'softmax':
                self.layers.append(nn.Softmax(dim=1).to(self.device))
            if size is not None:
                prev_size = size
        
        # Output layer
        self.layers.append(nn.Linear(prev_size, num_classes).to(self.device))
        self.layers.append(nn.Softmax(dim=1).to(self.device))
  
    def forward(self, x):
        x = x.to(self.device)
        for layer in self.layers:
            x = layer(x)
        return x

In [237]:
layers_list = [
    ('linear', 128),
    ('relu', None),
    ('linear', 64),
    ('relu', None),
    ('linear', 32),
    ('relu', None),
    ('linear', 16),
    ('relu', None),
    ('linear', 8),
]

In [180]:
#Training
batch_size=128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [170]:
train_loader.batch_size

128

In [171]:
train_loader

In [172]:
for sampleimg, samplelbl in train_loader:
    print(sampleimg.shape)
    print(samplelbl)
    break

torch.Size([128, 1, 28, 28])
tensor([7, 0, 5, 7, 1, 8, 1, 0, 2, 2, 6, 9, 8, 3, 0, 9, 4, 7, 0, 6, 8, 7, 5, 6,
        7, 3, 4, 8, 1, 0, 7, 1, 2, 9, 6, 7, 5, 9, 0, 3, 5, 0, 4, 6, 1, 9, 0, 9,
        5, 8, 6, 5, 2, 1, 9, 3, 2, 1, 4, 6, 1, 8, 7, 7, 1, 8, 8, 5, 1, 1, 7, 3,
        9, 8, 0, 3, 4, 1, 1, 1, 7, 1, 3, 0, 0, 8, 2, 3, 1, 6, 9, 0, 0, 3, 6, 7,
        0, 1, 0, 4, 5, 2, 6, 3, 8, 8, 3, 9, 0, 1, 0, 3, 4, 8, 8, 2, 9, 2, 2, 6,
        6, 3, 6, 5, 0, 5, 9, 3])


In [173]:
nn.Flatten()(sampleimg).shape

torch.Size([128, 784])

In [174]:
sampleimg.view(-1, 28*28).shape

torch.Size([128, 784])

In [175]:
B, A,  T, C = sampleimg.shape

In [176]:
B, A,  T, C

(128, 1, 28, 28)

In [220]:
T*C

784

In [224]:
(max(train_dataset.targets)+1).item()

10

In [247]:
model = NN(input_size=T*C, num_classes= (max(train_dataset.targets)+1).item(),arch=layers_list).to(device)
model = model.to(device)

In [250]:
def calculate_accuracy(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    
    with torch.no_grad():  # Disable gradient calculation
        for images, labels in data_loader:
            images = images.view(-1, T*C).to(device)  # Flatten the image
            labels = labels.to(device)
            outputs = model(images)
            predicted = torch.argmax(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy

In [249]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
for iter in range(1000):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, T*C).to(device) #Flatten the image
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        # if (i+1) % 100 == 0:
        #     print(f'Iteration: {iter+1}, Batch={i+1}, Loss: {loss.item()}')
    if (iter)%5 ==0:
        train_accuracy = calculate_accuracy(model, train_loader, device)
        test_accuracy = calculate_accuracy(model, test_loader, device)
        print(f'Iteration: {iter+1}, Train Accuracy: {train_accuracy}, Test Accuracy: {test_accuracy}')

Iteration: 1, Train Accuracy: 72.28833333333333, Test Accuracy: 72.3
Iteration: 6, Train Accuracy: 92.92166666666667, Test Accuracy: 92.8
Iteration: 11, Train Accuracy: 94.97666666666667, Test Accuracy: 94.16
Iteration: 16, Train Accuracy: 96.205, Test Accuracy: 95.59
Iteration: 21, Train Accuracy: 96.71833333333333, Test Accuracy: 96.05
Iteration: 26, Train Accuracy: 96.71333333333334, Test Accuracy: 95.96
Iteration: 31, Train Accuracy: 97.23, Test Accuracy: 96.26
Iteration: 36, Train Accuracy: 96.98166666666667, Test Accuracy: 95.98
Iteration: 41, Train Accuracy: 97.84833333333333, Test Accuracy: 97.13
Iteration: 46, Train Accuracy: 96.54166666666667, Test Accuracy: 95.66
Iteration: 51, Train Accuracy: 97.48166666666667, Test Accuracy: 96.35
Iteration: 56, Train Accuracy: 97.10166666666667, Test Accuracy: 96.35
Iteration: 61, Train Accuracy: 97.69833333333334, Test Accuracy: 96.76
Iteration: 66, Train Accuracy: 97.645, Test Accuracy: 96.83
Iteration: 71, Train Accuracy: 97.2866666666

KeyboardInterrupt: 